In [1]:
using Pkg
Pkg.activate(".")
using Turing, CSV, EzXML, Glob, Dates, StatsPlots, HDF5, Measures, DataFrames, StatsBase, Logging
Logging.global_logger(Logging.SimpleLogger(stdout, Logging.Error))


  Activating project at `~/BayesCSF`


Base.CoreLogging.SimpleLogger(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(19) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Info, Dict{Any, Int64}())

In [2]:
include("Utils.jl")
# theme(:dracula; palette=palette(:seaborn_colorblind))
theme(:dracula)

datapath = "/Users/jjc/CSF/Recordings/"
path = pwd();
savepath = "/Users/jjc/CSF/"
files = glob("*.hdf5", datapath)

# fid = 278
# global Data = readCSF(files[fid])
global Data = readCSF("/Users/jjc/CSF/Recordings/inf_20180504120604.hdf5")
# global Data = readCSF("/Users/jjc/CSF/Recordings/inf_20170118113759_INF2.hdf5")
# global Data = readCSF("/Users/jjc/CSF/Recordings/inf_20101213150719_INF2.hdf5")

icp = Data["ICP"][Data["infusion_start_frame"]:Data["infusion_end_frame"]]
icp = icp[.~isnan.(icp)]

# Priors and bounds
global params_means = [10.45, 0.33, 7.5]
global params_stddevs = [2.03, 0.08, 1.5]
global lower_bounds = [0.01, 0.01, -10.0]
global upper_bounds = [50.0, 1.0, Data["P_b"]]
global Ib_max = 1.0
global Ib_min = 0.01

# Turing.jl settings
sampler = NUTS()
# sampler = MH()
num_samples = 100
num_chains = 4

# Sampling
x0 = [0.33, 7.5, 10.45, 1.0]
# chain = sample(curve_fitting(icp), sampler, MCMCThreads(), num_samples, num_chains, init_params=Iterators.repeated(x0))
chain = sample(curve_fitting(icp), sampler, MCMCThreads(), num_samples, num_chains);
;

Sampling (4 threads)   0%|                              |  ETA: N/A


Sampling (4 threads)  25%|███████▌                      |  ETA: 0:00:39
Sampling (4 threads)  50%|███████████████               |  ETA: 0:00:13
Sampling (4 threads)  75%|██████████████████████▌       |  ETA: 0:00:04
Sampling (4 threads) 100%|██████████████████████████████| Time: 0:00:13


Sampling (4 threads) 100%|██████████████████████████████| Time: 0:00:13


In [ ]:
res_summary = DataFrame(summarize(chain))
E_mean, P0_mean, Rout_mean = res_summary.mean[1:3]
Ib_chain = (Data["P_b"] .- chain[:P0]) ./ chain[:Rout]
Ib_mean = (Data["P_b"] - P0_mean) / Rout_mean

NRMSE_Bayes_mean = round(calc_model_plot(Ib_mean, E_mean, P0_mean, P0_mean)[2], digits=3)
NRMSE_GD = round(calc_model_plot(Data["I_b"], Data["E"], Data["P_0"], Data["P_0"])[2], digits=3)

plotmodel(Ib_mean, E_mean, P0_mean, P0_mean, zeros(3), zeros(3), "dark", "")

title!(
    "Resistance to CSF outflow = $(round(Rout_mean,digits=2)) ± $(round(std(chain[:Rout]), digits=2)) [mmHg/mL/min]\n" *
    "Elasticity coefficient = $(round(E_mean,digits=2)) ± $(round(std(chain[:E]), digits=2)) [1/mL]\n" *
    "Reference pressure = $(round(P0_mean,digits=2)) ± $(round(std(chain[:P0]), digits=2)) [mmHg]\n" *
    "CSF production rate = $(round(Ib_mean,digits=2)) ± $(round(std(Ib_chain), digits=2)) [mL/min]\n" *
    "Error (Bayesian) = $NRMSE_Bayes_mean\n" *
    "Error (Gradient descent) = $NRMSE_GD\n",
    grid=true,
    size=(700, 500),
    dpi=300,
    margin=10mm,
    legend=:topleft
)

In [ ]:
dicp = diff(icp)

moving_average(vs,n) = [sum(@view vs[i:(i+n-1)])/n for i in 1:(length(vs)-(n-1))]

icp = Data["ICP"][1:Data["plateau_end"]]
icp_smooth = moving_average(icp, 30)
d_icp_smooth = diff(icp_smooth)

# plot(dicp, icp[2:end], seriestype=:scatter, xlabel="dICP", ylabel="ICP")

plot(d_icp_smooth, icp_smooth[2:end], seriestype=:scatter, xlabel="dICP", ylabel="ICP")

X = hcat(d_icp_smooth, icp_smooth[2:end])


In [ ]:
using Clustering

res = kmeans(X', 3)

res_ass = res.assignments

sc = scatter(d_icp_smooth, icp_smooth, color=res_ass,
        xlabel="dICP", ylabel="ICP",
        title="K-means Clustering Results", legend=false)

lp = plot(icp_smooth, color=res_ass, xlabel="Time", ylabel="ICP", legend=false)


plot(sc, lp, layout=(2,1))

In [ ]:
plot(icp_smooth, color=res_ass)

In [ ]:
# using Pkg

ENV["PYTHON"] = "/opt/homebrew/Caskroom/miniforge/base/bin/python"
theme(:dracula; palette=palette(:seaborn_colorblind))
# Pkg.activate(".")
# Pkg.build("PyCall")
using HDBSCAN, Clustering, PyCall


res = hdbscan(X', min_cluster_size=15, min_samples=3)
res_ass = res.assignments

sc = scatter(d_icp_smooth, icp_smooth, color=res_ass,
    xlabel="dICP", ylabel="ICP",
    title="Clustering Results", legend=false)

lp = plot(icp_smooth, color=res_ass, xlabel="Time", ylabel="ICP", legend=false)


plot(sc, lp, layout=(2, 1))

